In [ ]:
!pip install textattack transformers pandas

Attack -  it is only useful for attacking one example at a time.

In [ ]:
import textattack
import transformers

# Load model, tokenizer, and model_wrapper
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# Construct our four components for `Attack`
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [
    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.9)
]
transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = textattack.Attack(goal_function, constraints, transformation, search_method)

input_text = "I really enjoyed the new movie that came out last month."
label = 1 #Positive
attack_result = attack.attack(input_text, label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attacker - provides support attacking multiple samples in parallel (i.e. multi-GPU), saving checkpoints, or logging results to text file, CSV file,

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
import textattack
import transformers

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")

# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(
    num_examples=20,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset() # Doesn't work out of the box needs to have ` Resource averaged_perceptron_tagger_eng` which can be loaded using the above code snippet

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split test.
textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


  5%|▌         | 1/20 [02:10<41:22, 130.66s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   5%|▌         | 1/20 [02:10<41:27, 130.92s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 2/20 [02:16<20:27, 68.22s/it] 
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  10%|█         | 2/20 [02:16<20:28, 68.23s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  15%|█▌        | 3/20 [02:19<13:10, 46.51s/it]
[Succeeded / Failed / Skipped / Total] 3 / 0 / 0 / 3:  15%|█▌        | 3/20 [02:19<13:10, 46.52s/it]
[Succeeded / Failed / Skipped / Total] 3 / 0 / 0 / 3:  20%|██        | 4/20 [02:59<11:58, 44.92s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:  20%|██        | 4/20 [02:59<11:58, 44.93s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:  25%|██▌       | 5/20 [02:59<08:59, 35.95s/it]textattack: Saving checkpoint under "checkpoints/1733081804863.ta.chkpt" at 2024-12-01 19:36:44 after 5 attacks.



[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:  30%|███       | 6/20 [03:10<07:23, 31.68s/it]
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:  30%|███       | 6/20 [03:10<07:23, 31.69s/it]
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:  35%|███▌      | 7/20 [03:20<06:13, 28.70s/it]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 1 / 7:  35%|███▌      | 7/20 [03:20<06:13, 28.70s/it]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 1 / 7:  40%|████      | 8/20 [03:54<05:51, 29.31s/it]
[Succeeded / Failed / Skipped / Total] 7 / 0 / 1 / 8:  40%|████      | 8/20 [03:54<05:51, 29.31s/it]
[Succeeded / Failed / Skipped / Total] 7 / 0 / 1 / 8:  45%|████▌     | 9/20 [04:09<05:04, 27.69s/it]
[Succeeded / Failed / Skipped / Total] 8 / 0 / 1 / 9:  45%|████▌     | 9/20 [04:09<05:04, 27.69s/it]
[Succeeded / Failed / Skipped / Total] 8 / 0 / 1 / 9:  50%|█████     | 10/20 [05:09<05:09, 30.98s/it]
[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:  50%|█████     | 10/20 [05:09<05:0


[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:  55%|█████▌    | 11/20 [05:33<04:33, 30.35s/it]
[Succeeded / Failed / Skipped / Total] 10 / 0 / 1 / 11:  55%|█████▌    | 11/20 [05:33<04:33, 30.36s/it]
[Succeeded / Failed / Skipped / Total] 10 / 0 / 1 / 11:  60%|██████    | 12/20 [05:40<03:46, 28.37s/it]
[Succeeded / Failed / Skipped / Total] 11 / 0 / 1 / 12:  60%|██████    | 12/20 [05:40<03:46, 28.37s/it]
[Succeeded / Failed / Skipped / Total] 11 / 0 / 1 / 12:  65%|██████▌   | 13/20 [06:08<03:18, 28.38s/it]
[Succeeded / Failed / Skipped / Total] 12 / 0 / 1 / 13:  65%|██████▌   | 13/20 [06:08<03:18, 28.38s/it]
[Succeeded / Failed / Skipped / Total] 12 / 0 / 1 / 13:  70%|███████   | 14/20 [06:20<02:43, 27.17s/it]
[Succeeded / Failed / Skipped / Total] 13 / 0 / 1 / 14:  70%|███████   | 14/20 [06:20<02:43, 27.17s/it]
[Succeeded / Failed / Skipped / Total] 13 / 0 / 1 / 14:  75%|███████▌  | 15/20 [06:38<02:12, 26.56s/it]
[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:  75%|███


[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:  80%|████████  | 16/20 [06:44<01:41, 25.31s/it]
[Succeeded / Failed / Skipped / Total] 15 / 0 / 1 / 16:  80%|████████  | 16/20 [06:45<01:41, 25.31s/it]
[Succeeded / Failed / Skipped / Total] 15 / 0 / 1 / 16:  85%|████████▌ | 17/20 [06:54<01:13, 24.40s/it]
[Succeeded / Failed / Skipped / Total] 16 / 0 / 1 / 17:  85%|████████▌ | 17/20 [06:54<01:13, 24.40s/it]
[Succeeded / Failed / Skipped / Total] 16 / 0 / 1 / 17:  90%|█████████ | 18/20 [07:02<00:46, 23.47s/it]
[Succeeded / Failed / Skipped / Total] 17 / 0 / 1 / 18:  90%|█████████ | 18/20 [07:02<00:46, 23.47s/it]
[Succeeded / Failed / Skipped / Total] 17 / 0 / 2 / 19:  95%|█████████▌| 19/20 [07:02<00:22, 22.24s/it]
[Succeeded / Failed / Skipped / Total] 17 / 0 / 2 / 19: 100%|██████████| 20/20 [07:25<00:00, 22.30s/it]
[Succeeded / Failed / Skipped / Total] 18 / 0 / 2 / 20: 100%|██████████| 20/20 [07:25<00:00, 22.30s/it]textattack: Saving checkpoint under "checkpoints/1733082071062.t





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 18     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 11.04% |
| Average num. words per input: | 203.75 |
| Avg num queries:              | 624.67 |
+-------------------------------+--------+


Adversarial Training

In [ ]:
import textattack
import transformers

model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# We only use DeepWordBugGao2018 to demonstration purposes.
attack = textattack.attack_recipes.DeepWordBugGao2018.build(model_wrapper)
train_dataset = textattack.datasets.HuggingFaceDataset("imdb", split="train")
eval_dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")

# Train for 3 epochs with 1 initial clean epochs, 1000 adversarial examples per epoch, learning rate of 5e-5, and effective batch size of 32 (8x4).
training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=1000,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    log_to_tb=True,
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    train_dataset,
    eval_dataset,
    training_args
)
trainer.train()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split train.
textattack: Loading datasets dataset imdb, split test.
textattack: Writing logs to ./outputs/2024-12-01-19-41-28-836128/train_log.txt.
textattack: Wrote original training args to ./outputs/2024-12-01-19-41-28-836128/training_args.json.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 25000
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattac

Loss 0.26010:  32%|███▏      | 1029/3250 [12:40<27:23,  1.35it/s]